# UFC FROM BRZ TO SLVR

1) Imports and Initialize Spark Session

In [1]:
#Import neccesary libraries
from pyspark.sql.functions import col, sum
from pyspark.sql.window import Window
from pyspark.sql.functions import avg, col, when, count, expr
from pyspark.ml.feature import StringIndexer, OneHotEncoder
from pyspark.ml import Pipeline
import matplotlib.pyplot as plt
import seaborn as sns
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.stat import Correlation
from pyspark.sql.functions import regexp_extract, col, expr
from pyspark.sql.functions import regexp_replace
from pyspark.sql import functions as F

In [2]:
import pandas as pd
pd.set_option('display.max_colwidth', None)
import numpy as np
np.bool = np.bool_
#current notebook name
notebook_name = __session__.replace('.ipynb','')[__session__.rfind('/')+1:] 
# HDFS base paths
hdfs_lakehouse_base_path = 'hdfs://localhost:9000/lakehouse/'
hdfs_warehouse_base_path = 'hdfs://localhost:9000/warehouse'

In [3]:
import os
dependencies = ["org.apache.spark:spark-avro_2.12:3.5.0",
                "io.delta:delta-iceberg_2.12:3.0.0"]
os.environ['PYSPARK_SUBMIT_ARGS']= f"--packages {','.join(dependencies)} pyspark-shell"
os.environ['PYARROW_IGNORE_TIMEZONE'] = 'true'

In [4]:
from pyspark.sql.session import SparkSession

spark = (SparkSession.builder
    .appName(notebook_name)
    .config("spark.log.level","ERROR")
    .config("spark.sql.warehouse.dir",hdfs_warehouse_base_path)
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
    .enableHiveSupport()
    .getOrCreate()
)

24/12/16 01:24:45 WARN Utils: Your hostname, osbdet resolves to a loopback address: 127.0.0.1; using 10.0.2.15 instead (on interface enp0s1)
24/12/16 01:24:45 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Ivy Default Cache set to: /home/osbdet/.ivy2/cache
The jars for the packages stored in: /home/osbdet/.ivy2/jars
org.apache.spark#spark-avro_2.12 added as a dependency
io.delta#delta-iceberg_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-6b8a6ab8-a3ae-4cd7-b653-ac4b676444b3;1.0
	confs: [default]
	found org.apache.spark#spark-avro_2.12;3.5.0 in central
	found org.tukaani#xz;1.9 in central


:: loading settings :: url = jar:file:/home/osbdet/.jupyter_venv/lib/python3.11/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


	found io.delta#delta-iceberg_2.12;3.0.0 in central
	found io.delta#delta-spark_2.12;3.0.0 in central
	found io.delta#delta-storage;3.0.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
	found org.scala-lang.modules#scala-collection-compat_2.12;2.1.1 in central
	found com.github.ben-manes.caffeine#caffeine;2.9.3 in central
	found org.checkerframework#checker-qual;3.19.0 in central
	found com.google.errorprone#error_prone_annotations;2.10.0 in central
:: resolution report :: resolve 195ms :: artifacts dl 10ms
	:: modules in use:
	com.github.ben-manes.caffeine#caffeine;2.9.3 from central in [default]
	com.google.errorprone#error_prone_annotations;2.10.0 from central in [default]
	io.delta#delta-iceberg_2.12;3.0.0 from central in [default]
	io.delta#delta-spark_2.12;3.0.0 from central in [default]
	io.delta#delta-storage;3.0.0 from central in [default]
	org.antlr#antlr4-runtime;4.9.3 from central in [default]
	org.apache.spark#spark-avro_2.12;3.5.0 from central in [default]
	o

2. Load and Understand Datasets

In [5]:
#fighters
fighters_df = (spark.read
              .option("header","true")
              .option("inferSchema", "true")
              .csv(f"{hdfs_lakehouse_base_path}/bronze/UFC/UFC-fighters"))

In [6]:
fighters_df.printSchema()

root
 |-- url: string (nullable = true)
 |-- fighter_name: string (nullable = true)
 |-- nickname: string (nullable = true)
 |-- birth_date: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- death_date: string (nullable = true)
 |-- location: string (nullable = true)
 |-- country: string (nullable = true)
 |-- height: string (nullable = true)
 |-- weight: string (nullable = true)
 |-- association: string (nullable = true)
 |-- weight_class: string (nullable = true)
 |-- wins: integer (nullable = true)
 |-- wins_ko: integer (nullable = true)
 |-- wins_submission: integer (nullable = true)
 |-- wins_decision: integer (nullable = true)
 |-- wins_other: integer (nullable = true)
 |-- lossess: integer (nullable = true)
 |-- losses_ko: integer (nullable = true)
 |-- losses_submission: integer (nullable = true)
 |-- losses_decision: integer (nullable = true)
 |-- losses_other: integer (nullable = true)



In [6]:
fighters_df.toPandas().head()

,url,fighter_name,nickname,birth_date,age,death_date,location,country,height,weight,...,wins,wins_ko,wins_submission,wins_decision,wins_other,lossess,losses_ko,losses_submission,losses_decision,losses_other
0,/fighter/Kharun-Atlangeriev-167431,Kharun Atlangeriev,Predator,05/10/1992,28.0,None,None,Russia,"5'10""""",155 lbs,...,11,5,5,1,NaN,11,0,1,1,NaN
1,/fighter/Shojin-Miki-157565,Shojin Miki,Mini Margarito,None,NaN,None,"Honolulu, Hawaii",United States,"5'9""""",125 lbs,...,6,2,4,0,NaN,6,2,0,5,NaN
2,/fighter/Ryan-Keenan-33827,Ryan Keenan,Who Dat?,15/06/1985,36.0,None,"Windermere, Florida",United States,"6'0""""",155 lbs,...,6,3,3,0,NaN,6,1,1,0,NaN
3,/fighter/Dustin-Jacoby-73825,Dustin Jacoby,The Hanyak,04/04/1988,33.0,None,"Arenzville, Illinois",United States,"6'4""""",205 lbs,...,14,9,1,4,NaN,14,1,2,2,NaN
4,/fighter/Richardson-Moreira-72825,Richardson Moreira,Rick Monstro,31/03/1984,37.0,None,"Campinas, Sao Paulo",Brazil,"6'2""""",205 lbs,...,8,2,5,1,NaN,8,1,1,3,NaN


In [7]:
upcoming_df=(spark.read
              .option("header","true")
              .option("inferSchema", "true")
              .csv(f"{hdfs_lakehouse_base_path}/bronze/UFC/UFC-fights/upcoming_fights"))

In [8]:
upcoming_df.printSchema()

root
 |-- RedFighter: string (nullable = true)
 |-- BlueFighter: string (nullable = true)
 |-- RedOdds: integer (nullable = true)
 |-- BlueOdds: integer (nullable = true)
 |-- RedExpectedValue: double (nullable = true)
 |-- BlueExpectedValue: double (nullable = true)
 |-- Date: date (nullable = true)
 |-- Location: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Winner: string (nullable = true)
 |-- TitleBout: boolean (nullable = true)
 |-- WeightClass: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- NumberOfRounds: integer (nullable = true)
 |-- BlueCurrentLoseStreak: integer (nullable = true)
 |-- BlueCurrentWinStreak: integer (nullable = true)
 |-- BlueDraws: integer (nullable = true)
 |-- BlueAvgSigStrLanded: double (nullable = true)
 |-- BlueAvgSigStrPct: double (nullable = true)
 |-- BlueAvgSubAtt: double (nullable = true)
 |-- BlueAvgTDLanded: double (nullable = true)
 |-- BlueAvgTDPct: double (nullable = true)
 |-- BlueLongestWinStreak: 

In [8]:
upcoming_df.toPandas().head()

,RedFighter,BlueFighter,RedOdds,BlueOdds,RedExpectedValue,BlueExpectedValue,Date,Location,Country,Winner,...,FinishDetails,FinishRound,FinishRoundTime,TotalFightTimeSecs,RedDecOdds,BlueDecOdds,RSubOdds,BSubOdds,RKOOdds,BKOOdds
0,Alexandre Pantoja,Kai Asakura,-250,215,40.0000,215.0,2024-12-07,"Las Vegas, Nevada, USA",USA,None,...,None,None,None,None,300,800,150,2500,400,350
1,Shavkat Rakhmonov,Ian Machado Garry,-210,295,47.6190,295.0,2024-12-07,"Las Vegas, Nevada, USA",USA,None,...,None,None,None,None,250,650,180,3000,240,700
2,Ciryl Gane,Alexander Volkov,-380,300,26.3158,300.0,2024-12-07,"Las Vegas, Nevada, USA",USA,None,...,None,None,None,None,-160,450,1100,3000,350,1100
3,Bryce Mitchell,Kron Gracie,-950,625,10.5263,625.0,2024-12-07,"Las Vegas, Nevada, USA",USA,None,...,None,None,None,None,-200,1100,380,1400,500,4000
4,Nate Landwehr,Dooho Choi,-130,110,76.9231,110.0,2024-12-07,"Las Vegas, Nevada, USA",USA,None,...,None,None,None,None,275,550,500,700,300,250


In [10]:
history_df=(spark.read
              .option("header","true")
              .option("inferSchema", "true")
              .csv(f"{hdfs_lakehouse_base_path}/bronze/UFC/UFC-fights/previous_fights"))

In [11]:
history_df.printSchema()

root
 |-- RedFighter: string (nullable = true)
 |-- BlueFighter: string (nullable = true)
 |-- RedOdds: double (nullable = true)
 |-- BlueOdds: double (nullable = true)
 |-- RedExpectedValue: double (nullable = true)
 |-- BlueExpectedValue: double (nullable = true)
 |-- Date: date (nullable = true)
 |-- Location: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Winner: string (nullable = true)
 |-- TitleBout: boolean (nullable = true)
 |-- WeightClass: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- NumberOfRounds: integer (nullable = true)
 |-- BlueCurrentLoseStreak: integer (nullable = true)
 |-- BlueCurrentWinStreak: integer (nullable = true)
 |-- BlueDraws: integer (nullable = true)
 |-- BlueAvgSigStrLanded: double (nullable = true)
 |-- BlueAvgSigStrPct: double (nullable = true)
 |-- BlueAvgSubAtt: double (nullable = true)
 |-- BlueAvgTDLanded: double (nullable = true)
 |-- BlueAvgTDPct: double (nullable = true)
 |-- BlueLongestWinStreak: in

In [10]:
history_df.toPandas().head()

,RedFighter,BlueFighter,RedOdds,BlueOdds,RedExpectedValue,BlueExpectedValue,Date,Location,Country,Winner,...,FinishDetails,FinishRound,FinishRoundTime,TotalFightTimeSecs,RedDecOdds,BlueDecOdds,RSubOdds,BSubOdds,RKOOdds,BKOOdds
0,Petr Yan,Deiveson Figueiredo,-305.0,245.0,32.7869,245.0000,2024-11-23,"Macau, China",China,Red,...,None,5.0,2024-12-15 05:00:00,1500.0,110.0,450.0,2000.0,1600.0,200.0,1000.0
1,Yan Xiaonan,Tabatha Ricci,-198.0,164.0,50.5051,164.0000,2024-11-23,"Macau, China",China,Red,...,None,3.0,2024-12-15 05:00:00,900.0,110.0,215.0,2000.0,1100.0,200.0,1800.0
2,Song Kenan,Muslim Salikhov,NaN,-230.0,NaN,43.4783,2024-11-23,"Macau, China",China,Blue,...,Kick,1.0,2024-12-15 03:49:00,229.0,NaN,175.0,NaN,1800.0,NaN,200.0
3,Wang Cong,Gabriella Fernandes,-950.0,750.0,10.5263,750.0000,2024-11-23,"Macau, China",China,Blue,...,Rear Naked Choke,2.0,2024-12-15 03:49:00,529.0,-110.0,1000.0,460.0,3000.0,280.0,2200.0
4,Volkan Oezdemir,Carlos Ulberg,215.0,-265.0,215.0000,37.7358,2024-11-23,"Macau, China",China,Blue,...,None,3.0,2024-12-15 05:00:00,900.0,500.0,240.0,1600.0,800.0,500.0,130.0


In [12]:
rankings_df=(spark.read
              .option("header","true")
              .option("inferSchema", "true")
              .csv(f"{hdfs_lakehouse_base_path}/bronze/UFC/UFC-rankings"))

In [13]:
rankings_df.printSchema()

root
 |-- date: date (nullable = true)
 |-- weightclass: string (nullable = true)
 |-- fighter: string (nullable = true)
 |-- rank: integer (nullable = true)



In [12]:
rankings_df.toPandas().head()

,date,weightclass,fighter,rank
0,2013-02-04,Pound-for-Pound,Anderson Silva,1
1,2013-02-04,Pound-for-Pound,Jon Jones,2
2,2013-02-04,Pound-for-Pound,Georges St. Pierre,3
3,2013-02-04,Pound-for-Pound,José Aldo,4
4,2013-02-04,Pound-for-Pound,Benson Henderson,5


3. Data Cleaning

In [13]:
# Function to identify columns with all null values
def show_all_null_columns(df, dataset_name):
    # Calculate total rows in the DataFrame
    total_rows = df.count()
    
    # Calculate the number of null values in each column
    null_counts = df.select([
        sum(col(c).isNull().cast("int")).alias(c) for c in df.columns
    ])
    
    # Identify columns where the number of nulls is equal to the total number of rows
    columns_all_null = [col_name for col_name, count in null_counts.first().asDict().items() if count == total_rows]
    
    # Display columns with all null values
    if columns_all_null:
        print(f"Columns with all null values in {dataset_name}: {columns_all_null}")
    else:
        print(f"No columns with all null values found in {dataset_name}")

# Apply the function to each dataset
show_all_null_columns(fighters_df, "Fighters Dataset")
show_all_null_columns(upcoming_df, "Upcoming Fights Dataset")
show_all_null_columns(history_df, "Historical Fights Dataset")
show_all_null_columns(rankings_df, "Rankings Dataset")

No columns with all null values found in Fighters Dataset
Columns with all null values in Upcoming Fights Dataset: ['Winner', 'EmptyArena', 'RWFlyweightRank', 'RWFeatherweightRank', 'RWStrawweightRank', 'RWBantamweightRank', 'RMiddleweightRank', 'RLightweightRank', 'RBantamweightRank', 'BWFlyweightRank', 'BWFeatherweightRank', 'BWStrawweightRank', 'BWBantamweightRank', 'BMiddleweightRank', 'BLightweightRank', 'BBantamweightRank', 'BFlyweightRank', 'BPFPRank', 'Finish', 'FinishDetails', 'FinishRound', 'FinishRoundTime', 'TotalFightTimeSecs']
No columns with all null values found in Historical Fights Dataset
No columns with all null values found in Rankings Dataset


No columns in relevant datasets with all null values, so no entire columns will be dropped right now. Mising values in Upcoming fights dataset are what we aim to predict so we will analyze the value of these columns and decide if they are critical or not then drop them.

In [14]:
def check_missing_values(df, dataset_name):
    print(f"\nMissing Values in {dataset_name}:")
    df.select([sum(col(c).isNull().cast("int")).alias(c) for c in df.columns]).show()

check_missing_values(fighters_df, "Fighters Dataset")
check_missing_values(upcoming_df, "Upcoming Fights Dataset")
check_missing_values(history_df, "Historical Fights Dataset")
check_missing_values(rankings_df, "Rankings Dataset")


Missing Values in Fighters Dataset:
+---+------------+--------+----------+----+----------+--------+-------+------+------+-----------+------------+----+-------+---------------+-------------+----------+-------+---------+-----------------+---------------+------------+
|url|fighter_name|nickname|birth_date| age|death_date|location|country|height|weight|association|weight_class|wins|wins_ko|wins_submission|wins_decision|wins_other|lossess|losses_ko|losses_submission|losses_decision|losses_other|
+---+------------+--------+----------+----+----------+--------+-------+------+------+-----------+------------+----+-------+---------------+-------------+----------+-------+---------+-----------------+---------------+------------+
|  0|           3|    1946|      1115|1149|      5114|     512|     22|     0|     0|        727|          30|   0|      0|              0|            0|      4722|      0|        0|                0|              0|        4722|
+---+------------+--------+----------+----+

For starters in missing values we are gonna drop rows with critical missing values, meaning values that are essential to our
analysis and modeling. That if missing it would impact the integrity of our data. This only applies to the fighters dataset as it is the only one missing critical values.

We have defined these as our critical values:
1. Fighter Identification
- fighter_name
- Country
2. Physical Attributes
- height
- weight
3. Performance Metrics
- wins
- lossess
4. Age and Birthdate
- age
- birth_date

In [15]:
# Define critical columns for each dataset
critical_columns_fighters = ["fighter_name", "country", "height", "weight", "wins", "lossess", "age", "birth_date"]

In [16]:
# Function to drop rows with critical missing values
def drop_critical_missing_values(df, critical_columns, dataset_name):
    print(f"\nChecking Missing Values in {dataset_name} Critical Columns:")
    df.select([
        sum(col(c).isNull().cast("int")).alias(c) for c in critical_columns
    ]).show()
    
    # Drop rows with missing values in critical columns
    df_clean = df.dropna(subset=critical_columns)
    
    print(f"{dataset_name} after dropping rows with critical missing values:")
    df_clean.show(5)
    
    return df_clean

In [17]:
fighters_df_clean = drop_critical_missing_values(fighters_df, critical_columns_fighters, "Fighters Dataset")


Checking Missing Values in Fighters Dataset Critical Columns:
+------------+-------+------+------+----+-------+----+----------+
|fighter_name|country|height|weight|wins|lossess| age|birth_date|
+------------+-------+------+------+----+-------+----+----------+
|           3|     22|     0|     0|   0|      0|1149|      1115|
+------------+-------+------+------+----+-------+----+----------+

Fighters Dataset after dropping rows with critical missing values:
+--------------------+------------------+------------+----------+---+----------+--------------------+-------------+------+-------+---------------+-----------------+----+-------+---------------+-------------+----------+-------+---------+-----------------+---------------+------------+
|                 url|      fighter_name|    nickname|birth_date|age|death_date|            location|      country|height| weight|    association|     weight_class|wins|wins_ko|wins_submission|wins_decision|wins_other|lossess|losses_ko|losses_submission|l

Remove Duplicates

In [18]:
# Function to check and remove duplicates
def remove_duplicates(df, dataset_name):
    print(f"\nChecking for duplicates in {dataset_name}:")
    initial_count = df.count()
    print(f"Initial count: {initial_count}")

    # Check for duplicate rows
    duplicates = df.groupBy(df.columns).count().filter("count > 1")
    if duplicates.count() > 0:
        print("Duplicate rows found:")
        duplicates.show()
    else:
        print("No duplicate rows found.")

    # Remove duplicates
    df_clean = df.dropDuplicates()

    final_count = df_clean.count()
    print(f"Final count after removing duplicates: {final_count}")
    print(f"Number of duplicates removed: {initial_count - final_count}")

    return df_clean

# Apply the function to each dataset
fighters_df_clean = remove_duplicates(fighters_df_clean, "Fighters Dataset")
upcoming_df_clean = remove_duplicates(upcoming_df, "Upcoming Fights Dataset")
history_df_clean = remove_duplicates(history_df, "Historical Fights Dataset")
rankings_df_clean = remove_duplicates(rankings_df, "Rankings Dataset")


Checking for duplicates in Fighters Dataset:
Initial count: 3998
No duplicate rows found.
Final count after removing duplicates: 3998
Number of duplicates removed: 0

Checking for duplicates in Upcoming Fights Dataset:
Initial count: 14
No duplicate rows found.
Final count after removing duplicates: 14
Number of duplicates removed: 0

Checking for duplicates in Historical Fights Dataset:
Initial count: 6514
No duplicate rows found.
Final count after removing duplicates: 6514
Number of duplicates removed: 0

Checking for duplicates in Rankings Dataset:
Initial count: 86011
No duplicate rows found.
Final count after removing duplicates: 86011
Number of duplicates removed: 0


Removing Unncecesary Columns to our Analysis

In [19]:
#Columns to drop
fighters_columns_to_drop = [
    "url",
    "death_date",
    "losses_other",
    "wins_other"
]

upcoming_columns_to_drop = [
    "EmptyArena",
    "BMatchWCRank",
    "RMatchWCRank",
    "RWFlyweightRank",
    "RWFeatherweightRank",
    "RWStrawweightRank",
    "RWBantamweightRank",
    "RHeavyweightRank",
    "RLightHeavyweightRank",
    "RMiddleweightRank",
    "RWelterweightRank",
    "RLightweightRank",
    "RFeatherweightRank",
    "RBantamweightRank",
    "RFlyweightRank",
    "RPFPRank",
    "BWFlyweightRank",
    "BWFeatherweightRank",
    "BWStrawweightRank",
    "BWBantamweightRank",
    "BHeavyweightRank",
    "BLightHeavyweightRank",
    "BMiddleweightRank",
    "BWelterweightRank",
    "BLightweightRank",
    "BFeatherweightRank",
    "BBantamweightRank",
    "BFlyweightRank",
    "BPFPRank",
    "BetterRank",
    "Finish",
    "FinishDetails",
    "FinishRound",
    "FinishRoundTime",
    "TotalFightTimeSecs"
]

history_columns_to_drop = [
    "RedExpectedValue",
    "BlueExpectedValue",
    "EmptyArena",
    "BMatchWCRank",
    "RMatchWCRank",
    "RWFlyweightRank",
    "RWFeatherweightRank",
    "RWStrawweightRank",
    "RWBantamweightRank",
    "RHeavyweightRank",
    "RLightHeavyweightRank",
    "RMiddleweightRank",
    "RWelterweightRank",
    "RLightweightRank",
    "RFeatherweightRank",
    "RBantamweightRank",
    "RFlyweightRank",
    "RPFPRank",
    "BWFlyweightRank",
    "BWFeatherweightRank",
    "BWStrawweightRank",
    "BWBantamweightRank",
    "BHeavyweightRank",
    "BLightHeavyweightRank",
    "BMiddleweightRank",
    "BWelterweightRank",
    "BLightweightRank",
    "BFeatherweightRank",
    "BBantamweightRank",
    "BFlyweightRank",
    "BPFPRank"
]

Decided to not drop any columns for rankings as all columns are important to our analysis and don't have a significant amount of missing values.

In [20]:
# Drop unnecessary columns
fighters_df_clean = fighters_df.drop(*fighters_columns_to_drop)
upcoming_df_clean = upcoming_df.drop(*upcoming_columns_to_drop)
history_df_clean = history_df.drop(*history_columns_to_drop)

# Show the cleaned DataFrames
print("Cleaned Fighters Dataset:")
fighters_df_clean.show(5)

print("Cleaned Upcoming Fights Dataset:")
upcoming_df_clean.show(5)

print("Cleaned Historical Fights Dataset:")
history_df_clean.show(5)

Cleaned Fighters Dataset:
+------------------+--------------+----------+----+--------------------+-------------+------+-------+--------------+-----------------+----+-------+---------------+-------------+-------+---------+-----------------+---------------+
|      fighter_name|      nickname|birth_date| age|            location|      country|height| weight|   association|     weight_class|wins|wins_ko|wins_submission|wins_decision|lossess|losses_ko|losses_submission|losses_decision|
+------------------+--------------+----------+----+--------------------+-------------+------+-------+--------------+-----------------+----+-------+---------------+-------------+-------+---------+-----------------+---------------+
|Kharun Atlangeriev|      Predator|05/10/1992|  28|                NULL|       Russia|5'10""|155 lbs|      OFS Team|      Lightweight|  11|      5|              5|            1|     11|        0|                1|              1|
|       Shojin Miki|Mini Margarito|      NULL|NULL|   

Checking datasets for data consistency

In [21]:
check_missing_values(fighters_df_clean, "Fighters Dataset")
check_missing_values(upcoming_df_clean, "Upcoming Fights Dataset")
check_missing_values(history_df_clean, "Historical Fights Dataset")
check_missing_values(rankings_df_clean, "Rankings Dataset")


Missing Values in Fighters Dataset:
+------------+--------+----------+----+--------+-------+------+------+-----------+------------+----+-------+---------------+-------------+-------+---------+-----------------+---------------+
|fighter_name|nickname|birth_date| age|location|country|height|weight|association|weight_class|wins|wins_ko|wins_submission|wins_decision|lossess|losses_ko|losses_submission|losses_decision|
+------------+--------+----------+----+--------+-------+------+------+-----------+------------+----+-------+---------------+-------------+-------+---------+-----------------+---------------+
|           3|    1946|      1115|1149|     512|     22|     0|     0|        727|          30|   0|      0|              0|            0|      0|        0|                0|              0|
+------------+--------+----------+----+--------+-------+------+------+-----------+------------+----+-------+---------------+-------------+-------+---------+-----------------+---------------+


Missin

There are still a lot of missing values in a lot of the columns of the datasets, we're gonna use different techniques now to get 
rid of them. Right now we are using the median to fill in the missing values from the "columns_to_fill"

1) Historical fights

In [22]:
columns_to_fill = [
    "BlueAvgSigStrLanded", "BlueAvgSigStrPct", "BlueAvgSubAtt", "BlueAvgTDLanded", "BlueAvgTDPct",
    "RedAvgSigStrLanded", "RedAvgSigStrPct", "RedAvgSubAtt", "RedAvgTDLanded", "RedAvgTDPct", "RedDecOdds",
    "BlueDecOdds", "RSubOdds", "RKOOdds", "BKOOdds", "TotalFightTimeSecs", "BSubOdds", "RedOdds", "BlueOdds"
]

# Function to calculate median for a specific column
def calculate_median(df, column_name):
    # Sort the column and calculate the approximate median
    median_value = df.approxQuantile(column_name, [0.5], 0.0)[0]
    return median_value

# Calculate medians for each column and fill missing values
for col_name in columns_to_fill:
    median = calculate_median(history_df_clean, col_name)
    print(f"Median for {col_name}: {median}")
    history_df_clean = history_df_clean.fillna({col_name: median})

Median for BlueAvgSigStrLanded: 9.5
Median for BlueAvgSigStrPct: 0.46
Median for BlueAvgSubAtt: 0.3
Median for BlueAvgTDLanded: 1.0
Median for BlueAvgTDPct: 0.33
Median for RedAvgSigStrLanded: 15.8333
Median for RedAvgSigStrPct: 0.46
Median for RedAvgSubAtt: 0.3333
Median for RedAvgTDLanded: 1.03
Median for RedAvgTDPct: 0.337
Median for RedDecOdds: 250.0
Median for BlueDecOdds: 350.0
Median for RSubOdds: 750.0
Median for RKOOdds: 420.0
Median for BKOOdds: 525.0
Median for TotalFightTimeSecs: 900.0
Median for BSubOdds: 1000.0
Median for RedOdds: -150.0
Median for BlueOdds: 130.0


In [23]:
check_missing_values(history_df_clean, "Historical Fights Dataset")


Missing Values in Historical Fights Dataset:
+----------+-----------+-------+--------+----+--------+-------+------+---------+-----------+------+--------------+---------------------+--------------------+---------+-------------------+----------------+-------------+---------------+------------+--------------------+----------+---------------------+-------------------+--------------------------+-----------------------+---------------------------+------------+--------------------+---------------------------+--------+----------+-------------+------------+-------------+--------------------+-------------------+--------+------------------+---------------+------------+--------------+-----------+-------------------+---------+--------------------+------------------+-------------------------+----------------------+--------------------------+-----------+-------------------+--------------------------+-------+---------+------------+-----------+------------+------+-------+-------------+------------+---

In [24]:
#Blue Stance - Fill with most common value

# Step 1: Calculate the most common value (mode) for the 'BlueStance' column
blue_stance_mode = (history_df_clean
                    .groupBy("BlueStance")
                    .agg(count("*").alias("count"))
                    .orderBy(col("count").desc())
                    .limit(1)
                    .collect()[0][0])

print(f"Most common value for BlueStance: {blue_stance_mode}")

# Step 2: Fill null values in 'BlueStance' with the most common value
history_df_clean = history_df_clean.fillna({"BlueStance": blue_stance_mode})

Most common value for BlueStance: Orthodox


In [25]:
check_missing_values(history_df_clean, "Historical Fights Dataset")


Missing Values in Historical Fights Dataset:
+----------+-----------+-------+--------+----+--------+-------+------+---------+-----------+------+--------------+---------------------+--------------------+---------+-------------------+----------------+-------------+---------------+------------+--------------------+----------+---------------------+-------------------+--------------------------+-----------------------+---------------------------+------------+--------------------+---------------------------+--------+----------+-------------+------------+-------------+--------------------+-------------------+--------+------------------+---------------+------------+--------------+-----------+-------------------+---------+--------------------+------------------+-------------------------+----------------------+--------------------------+-----------+-------------------+--------------------------+-------+---------+------------+-----------+------------+------+-------+-------------+------------+---

In [26]:
# Fill missing values with appropriate defaults
history_df_clean = history_df_clean.fillna({
    "Finish": "Decision",            # Default to 'Decision' if no finish occurred
    "FinishDetails": "N/A",          # 'N/A' for no specific finish details
    "FinishRound": 0,                # 0 to indicate no specific round (fight went the distance)
    "FinishRoundTime": "00:00"       # '00:00' for no specific finish time
})

In [27]:
check_missing_values(history_df_clean, "Historical Fights Dataset")

#history_df_clean now finally has no missing values!


Missing Values in Historical Fights Dataset:
+----------+-----------+-------+--------+----+--------+-------+------+---------+-----------+------+--------------+---------------------+--------------------+---------+-------------------+----------------+-------------+---------------+------------+--------------------+----------+---------------------+-------------------+--------------------------+-----------------------+---------------------------+------------+--------------------+---------------------------+--------+----------+-------------+------------+-------------+--------------------+-------------------+--------+------------------+---------------+------------+--------------+-----------+-------------------+---------+--------------------+------------------+-------------------------+----------------------+--------------------------+-----------+-------------------+--------------------------+-------+---------+------------+-----------+------------+------+-------+-------------+------------+---

2) Fighters dataset

In [28]:
# 1. Drop rows where 'fighter_name', 'birth_date', or 'age' are missing
fighters_df_clean = fighters_df.dropna(subset=["fighter_name", "birth_date", "age"])

# 2. Fill missing 'nickname' with "No Nickname"
fighters_df_clean = fighters_df_clean.fillna({"nickname": "No Nickname"})

# 3. Fill missing 'location', 'country', and 'association' with "Unknown"
fighters_df_clean = fighters_df_clean.fillna({
    "location": "Unknown",
    "country": "Unknown",
    "association": "Unknown"
})

# 4. Fill missing 'weight_class' based on 'weight'
# Example weight ranges mapped to weight classes
fighters_df_clean = fighters_df_clean.withColumn(
    "weight_class",
    when(col("weight").isNull(), "Unknown")
    .when(col("weight") <= 57, "Flyweight")
    .when((col("weight") > 57) & (col("weight") <= 61), "Bantamweight")
    .when((col("weight") > 61) & (col("weight") <= 66), "Featherweight")
    .when((col("weight") > 66) & (col("weight") <= 70), "Lightweight")
    .when((col("weight") > 70) & (col("weight") <= 77), "Welterweight")
    .when((col("weight") > 77) & (col("weight") <= 84), "Middleweight")
    .when((col("weight") > 84) & (col("weight") <= 93), "Light Heavyweight")
    .when((col("weight") > 93) & (col("weight") <= 120), "Heavyweight")
    .otherwise("Unknown")
)

# 5. Height -> From string to numerical values
# Extract feet and inches using regular expressions
fighters_df_clean = fighters_df_clean.withColumn("feet", regexp_extract(col("height"), r"(\d+)'", 1).cast("int"))
fighters_df_clean = fighters_df_clean.withColumn("inches", regexp_extract(col("height"), r"'(\d+)", 1).cast("int"))

# Convert to total inches
fighters_df_clean = fighters_df_clean.withColumn("height", (col("feet") * 12) + col("inches"))

# Drop the intermediate columns if desired
fighters_df_clean = fighters_df_clean.drop("feet", "inches")

# 6. Weight to numerical values
# Remove 'lbs' and convert to integer
fighters_df_clean = fighters_df_clean.withColumn("weight", regexp_replace(col("weight"), " lbs", "").cast("int"))


In [29]:
fighters_df_clean = fighters_df_clean.drop("death_date", "wins_other", "losses_other")

In [30]:
check_missing_values(fighters_df_clean, "Historical Fights Dataset")


Missing Values in Historical Fights Dataset:
+---+------------+--------+----------+---+--------+-------+------+------+-----------+------------+----+-------+---------------+-------------+-------+---------+-----------------+---------------+
|url|fighter_name|nickname|birth_date|age|location|country|height|weight|association|weight_class|wins|wins_ko|wins_submission|wins_decision|lossess|losses_ko|losses_submission|losses_decision|
+---+------------+--------+----------+---+--------+-------+------+------+-----------+------------+----+-------+---------------+-------------+-------+---------+-----------------+---------------+
|  0|           0|       0|         0|  0|       0|      0|     0|     0|          0|           0|   0|      0|              0|            0|      0|        0|                0|              0|
+---+------------+--------+----------+---+--------+-------+------+------+-----------+------------+----+-------+---------------+-------------+-------+---------+-----------------+-

In [31]:
check_missing_values(history_df_clean, "Historical Fights Dataset")
check_missing_values(upcoming_df_clean, "Upcoming Fights Dataset")
check_missing_values(rankings_df_clean, "Rankings Fights Dataset")


Missing Values in Historical Fights Dataset:
+----------+-----------+-------+--------+----+--------+-------+------+---------+-----------+------+--------------+---------------------+--------------------+---------+-------------------+----------------+-------------+---------------+------------+--------------------+----------+---------------------+-------------------+--------------------------+-----------------------+---------------------------+------------+--------------------+---------------------------+--------+----------+-------------+------------+-------------+--------------------+-------------------+--------+------------------+---------------+------------+--------------+-----------+-------------------+---------+--------------------+------------------+-------------------------+----------------------+--------------------------+-----------+-------------------+--------------------------+-------+---------+------------+-----------+------------+------+-------+-------------+------------+---

Now no datasets have any mising values - except Winner in Historical fight which is what we are trying to predict. SILVER STATUS

Handled Missing Values:

Critical values like fighter_name, birth_date, and age were managed by dropping rows.
Other fields like nickname, location, country, association, and weight_class were filled with appropriate defaults.
Removed Unnecessary Columns:

Dropped columns such as death_date, wins_other, and losses_other that weren't useful for analysis.
Filled Contextual Missing Values:

For history_df_clean, columns like BlueAvgSigStrLanded, RedAvgSigStrLanded, etc., were filled using the median to prevent outliers from affecting the data.
Columns related to fight finishes were filled with meaningful defaults.
Ensured Data Consistency:

Standardized string formats (e.g., uppercase).
Checked for and removed duplicates.
Ensured all datasets are clean and consistent.
Managed Target Column:

The Winner column in history_df_clean remains with missing values since it's the target variable for prediction.

<a id='2.4'></a>
###  Write DataFrames to Silver

In [32]:
spark.sql("DROP SCHEMA IF EXISTS ufc CASCADE")
spark.sql("CREATE SCHEMA IF NOT EXISTS ufc")

DataFrame[]

DataFrame[]

In [33]:
(fighters_df_clean.write
            .format("delta")
            .mode("overwrite")
            .option("path",f"{hdfs_lakehouse_base_path}/silver/UFC/UFC-fighters")
            .saveAsTable("ufc.fighters")
)

(history_df_clean.write
            .format("delta")
            .mode("overwrite")
            .option("path",f"{hdfs_lakehouse_base_path}/silver/UFC/UFC-fights/previous_fights")
            .saveAsTable("ufc.previous_fights")
)

(upcoming_df_clean.write
            .format("delta")
            .mode("overwrite")
            .option("path",f"{hdfs_lakehouse_base_path}/silver/UFC/UFC-fights/upcoming_fights")
            .saveAsTable("ufc.upcoming_fights")
)

(rankings_df_clean.write
            .format("delta")
            .mode("overwrite")
            .option("path",f"{hdfs_lakehouse_base_path}/silver/UFC/UFC-rankings")
            .saveAsTable("ufc.rankings")
)


In [34]:
%%sparksql
show databases

Execution time: 0.09 seconds


SparkSchemaWidget(nodes=(Node(close_icon='angle-down', close_icon_style='danger', icon='project-diagram', icon…

namespace
default
movielens
pokemon
ufc


In [35]:
%%sparksql
use ufc

Execution time: 0.03 seconds


SparkSchemaWidget(nodes=(Node(close_icon='angle-down', close_icon_style='danger', icon='project-diagram', icon…

""


In [36]:
%%sparksql
show tables

Execution time: 0.04 seconds


SparkSchemaWidget(nodes=(Node(close_icon='angle-down', close_icon_style='danger', icon='project-diagram', icon…

namespace,tableName,isTemporary
ufc,fighters,False
ufc,previous_fights,False
ufc,rankings,False
ufc,upcoming_fights,False


In [37]:
%%sparksql
select *
from ufc.fighters
limit 10

Execution time: 1.07 seconds


SparkSchemaWidget(nodes=(Node(close_icon='angle-down', close_icon_style='danger', icon='project-diagram', icon…

url,fighter_name,nickname,birth_date,age,location,country,height,weight,association,weight_class,wins,wins_ko,wins_submission,wins_decision,lossess,losses_ko,losses_submission,losses_decision
/fighter/Kharun-Atlangeriev-167431,Kharun Atlangeriev,Predator,05/10/1992,28,Unknown,Russia,70,155,OFS Team,Unknown,11,5,5,1,11,0,1,1
/fighter/Ryan-Keenan-33827,Ryan Keenan,Who Dat?,15/06/1985,36,"Windermere, Florida",United States,72,155,Gracie Orlando,Unknown,6,3,3,0,6,1,1,0
/fighter/Dustin-Jacoby-73825,Dustin Jacoby,The Hanyak,04/04/1988,33,"Arenzville, Illinois",United States,76,205,Factory X,Unknown,14,9,1,4,14,1,2,2
/fighter/Richardson-Moreira-72825,Richardson Moreira,Rick Monstro,31/03/1984,37,"Campinas, Sao Paulo",Brazil,74,205,Team Nogueira,Unknown,8,2,5,1,8,1,1,3
/fighter/Chris-Haseman-1449,Chris Haseman,The Hammer,02/06/1969,52,Unknown,Australia,71,198,Rings Australia,Unknown,20,2,14,3,20,8,3,6
/fighter/Fares-Ziam-184241,Fares Ziam,Smile Killer,21/03/1997,24,Lyon,France,72,155,Team Ezbiri,Unknown,12,5,4,3,12,0,2,1
/fighter/James-Bochnovic-130011,James Bochnovic,No Nickname,13/02/1989,32,"Kenosha, Wisconsin",United States,76,185,Rothwell MMA,Unknown,10,1,9,0,10,2,2,1
/fighter/Adrian-Pang-8427,Adrian Pang,The Hunter,02/09/1977,43,"Kokopo, East New Britain",Papua New Guinea,67,155,Integrated MMA,Unknown,23,6,10,7,23,3,1,9
/fighter/Haobin-Ma-214849,Haobin Ma,The Southern Eagle,23/01/1994,27,Unknown,China,0,125,Jincan Defense Club,Unknown,13,3,5,5,13,2,1,5
/fighter/Journey-Newson-77464,Journey Newson,No Nickname,07/03/1989,32,"Beaverton, Oregon",United States,65,135,Impact Jiu Jitsu,Unknown,9,3,3,3,9,2,0,1


24/12/15 02:48:52 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:124)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$$driverEndpoint(BlockManagerMasterEndpoint.scala:123)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.isExecutorAlive$lzycompute$1(BlockManagerMasterEndpoint.scala:688)
	at org.apache.spark.storage.BlockManagerMasterE